# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** Due to the complexity of the task and the amount of training time, I tried to keep the model simple. No change in the pretrained Resnet CNN was done apart from getting rid of the last linear layer to tranform the output. The RNN was also made using a simple LSTM with just the basic inputs.


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 224*224 is a suitable resolution for the task, so the transform_train was left as it is and the similar one was used for transform_test as well.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I did some research of what other people has done regarding this type of problem and chose the parameters so that they converge faster sacrificing accuracy (slight higher learning rate for example). The number of hidden size was gradually increased from 3 to 512 because increasing it seemed to lower perplexity and loss. 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** Adam optimizer because it works well pretty much universally

In [10]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import nltk
nltk.download('punkt')


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 32           # batch size
vocab_threshold = 4        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
          
# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 452/414113 [00:00<01:31, 4518.95it/s]

  0%|          | 848/414113 [00:00<01:35, 4333.18it/s]

Done (t=0.89s)
creating index...
index created!
Obtaining caption lengths...




  0%|          | 1306/414113 [00:00<01:33, 4402.42it/s]

  0%|          | 1751/414113 [00:00<01:33, 4414.75it/s]

  1%|          | 2201/414113 [00:00<01:32, 4438.96it/s]

  1%|          | 2650/414113 [00:00<01:32, 4450.88it/s]

  1%|          | 3093/414113 [00:00<01:32, 4443.26it/s]

  1%|          | 3526/414113 [00:00<01:33, 4406.22it/s]

  1%|          | 3958/414113 [00:00<01:33, 4378.89it/s]

  1%|          | 4408/414113 [00:01<01:32, 4413.67it/s]

  1%|          | 4837/414113 [00:01<01:34, 4330.58it/s]

  1%|▏         | 5295/414113 [00:01<01:32, 4400.30it/s]

  1%|▏         | 5730/414113 [00:01<01:33, 4373.38it/s]

  1%|▏         | 6183/414113 [00:01<01:32, 4416.82it/s]

  2%|▏         | 6626/414113 [00:01<01:32, 4419.27it/s]

  2%|▏         | 7067/414113 [00:01<01:35, 4264.09it/s]

  2%|▏         | 7501/414113 [00:01<01:34, 4284.59it/s]

  2%|▏         | 7957/414113 [00:01<01:33, 4362.74it/s]

  2%|▏         | 8418/414113 [00:01<01:31, 4433.24it/s]

  2%|▏         | 8873/414113 

 14%|█▍        | 59152/414113 [00:13<01:18, 4544.26it/s]

 14%|█▍        | 59611/414113 [00:13<01:17, 4557.52it/s]

 15%|█▍        | 60077/414113 [00:14<01:17, 4587.75it/s]

 15%|█▍        | 60537/414113 [00:14<01:19, 4426.96it/s]

 15%|█▍        | 60991/414113 [00:14<01:19, 4458.64it/s]

 15%|█▍        | 61457/414113 [00:14<01:18, 4516.65it/s]

 15%|█▍        | 61910/414113 [00:14<01:18, 4514.93it/s]

 15%|█▌        | 62363/414113 [00:14<01:17, 4512.85it/s]

 15%|█▌        | 62820/414113 [00:14<01:17, 4529.28it/s]

 15%|█▌        | 63274/414113 [00:14<01:17, 4531.72it/s]

 15%|█▌        | 63738/414113 [00:14<01:16, 4560.84it/s]

 16%|█▌        | 64198/414113 [00:14<01:16, 4570.12it/s]

 16%|█▌        | 64656/414113 [00:15<01:17, 4513.05it/s]

 16%|█▌        | 65117/414113 [00:15<01:16, 4540.39it/s]

 16%|█▌        | 65572/414113 [00:15<01:18, 4448.20it/s]

 16%|█▌        | 66020/414113 [00:15<01:18, 4456.28it/s]

 16%|█▌        | 66475/414113 [00:15<01:17, 4483.64it/s]

 16%|█▌       

 43%|████▎     | 179898/414113 [00:41<00:52, 4460.23it/s]


 58%|█████▊    | 240990/414113 [00:54<00:38, 4439.12it/s]


 73%|███████▎  | 301486/414113 [01:08<00:25, 4480.52it/s]


 87%|████████▋ | 362132/414113 [01:21<00:11, 4473.60it/s]


100%|██████████| 414113/414113 [01:33<00:00, 4422.50it/s]

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [11]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [1/12942], Loss: 9.0907, Perplexity: 8872.4974torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [2/12942], Loss: 9.0149, Perplexity: 8224.7805torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [3/12942], Loss: 8.8595, Perplexity: 7040.7930torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [4/12942], Loss: 8.4765, Perplexity: 4800.6162torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [5/12942], Loss: 7.6955, Perplexity: 2198.3780torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [6/12942], Loss: 6.9648, Perplexity: 1058.6546torch.Size([32, 15, 512])
torch.Size([32, 15, 8855])
Epoch [1/1], Step [7/12942], Loss: 6.2208, Perplexity: 503.1175torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [8/12942], Loss: 5.9713, Perplexity: 392.0245torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], 

Epoch [1/1], Step [71/12942], Loss: 4.7151, Perplexity: 111.6245torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [72/12942], Loss: 4.3707, Perplexity: 79.1025torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [73/12942], Loss: 4.3863, Perplexity: 80.3400torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [74/12942], Loss: 4.3212, Perplexity: 75.2801torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [75/12942], Loss: 4.6418, Perplexity: 103.7317torch.Size([32, 15, 512])
torch.Size([32, 15, 8855])
Epoch [1/1], Step [76/12942], Loss: 4.6927, Perplexity: 109.1452torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [77/12942], Loss: 4.7175, Perplexity: 111.8933torch.Size([32, 10, 512])
torch.Size([32, 10, 8855])
Epoch [1/1], Step [78/12942], Loss: 4.9875, Perplexity: 146.5637torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [79/12942], Loss: 4.4118, Perplexity: 82.4202torc

Epoch [1/1], Step [141/12942], Loss: 4.4479, Perplexity: 85.4451torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [142/12942], Loss: 4.1290, Perplexity: 62.1174torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [143/12942], Loss: 4.3734, Perplexity: 79.3136torch.Size([32, 15, 512])
torch.Size([32, 15, 8855])
Epoch [1/1], Step [144/12942], Loss: 4.2340, Perplexity: 68.9926torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [145/12942], Loss: 4.1662, Perplexity: 64.4726torch.Size([32, 19, 512])
torch.Size([32, 19, 8855])
Epoch [1/1], Step [146/12942], Loss: 4.5133, Perplexity: 91.2226torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [147/12942], Loss: 4.1339, Perplexity: 62.4180torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [148/12942], Loss: 4.1008, Perplexity: 60.3889torch.Size([32, 16, 512])
torch.Size([32, 16, 8855])
Epoch [1/1], Step [149/12942], Loss: 4.2860, Perplexity: 72.6753

Epoch [1/1], Step [211/12942], Loss: 3.7694, Perplexity: 43.3542torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [212/12942], Loss: 3.8211, Perplexity: 45.6549torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [213/12942], Loss: 4.0388, Perplexity: 56.7554torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [214/12942], Loss: 3.8981, Perplexity: 49.3104torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [215/12942], Loss: 3.7807, Perplexity: 43.8482torch.Size([32, 17, 512])
torch.Size([32, 17, 8855])
Epoch [1/1], Step [216/12942], Loss: 3.8671, Perplexity: 47.8034torch.Size([32, 18, 512])
torch.Size([32, 18, 8855])
Epoch [1/1], Step [217/12942], Loss: 4.4813, Perplexity: 88.3528torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [218/12942], Loss: 4.0686, Perplexity: 58.4770torch.Size([32, 10, 512])
torch.Size([32, 10, 8855])
Epoch [1/1], Step [219/12942], Loss: 4.0481, Perplexity: 57.2892

Epoch [1/1], Step [281/12942], Loss: 3.9372, Perplexity: 51.2736torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [282/12942], Loss: 3.9741, Perplexity: 53.2020torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [283/12942], Loss: 3.6825, Perplexity: 39.7461torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [284/12942], Loss: 3.7407, Perplexity: 42.1294torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [285/12942], Loss: 3.7457, Perplexity: 42.3370torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [286/12942], Loss: 3.7336, Perplexity: 41.8283torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [287/12942], Loss: 3.3295, Perplexity: 27.9248torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [288/12942], Loss: 3.6538, Perplexity: 38.6199torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [289/12942], Loss: 3.4999, Perplexity: 33.1111

...

Epoch [1/1], Step [12651/12942], Loss: 2.3319, Perplexity: 10.2978torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [12652/12942], Loss: 2.2373, Perplexity: 9.3680torch.Size([32, 10, 512])
torch.Size([32, 10, 8855])
Epoch [1/1], Step [12653/12942], Loss: 2.5055, Perplexity: 12.2501torch.Size([32, 10, 512])
torch.Size([32, 10, 8855])
Epoch [1/1], Step [12654/12942], Loss: 2.6603, Perplexity: 14.3003torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [12655/12942], Loss: 2.0853, Perplexity: 8.0471torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12656/12942], Loss: 2.1334, Perplexity: 8.4438torch.Size([32, 10, 512])
torch.Size([32, 10, 8855])
Epoch [1/1], Step [12657/12942], Loss: 2.5502, Perplexity: 12.8097torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12658/12942], Loss: 1.8250, Perplexity: 6.2025torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12659/12942], Loss: 1.9882, Perpl

Epoch [1/1], Step [12720/12942], Loss: 2.7179, Perplexity: 15.1484torch.Size([32, 10, 512])
torch.Size([32, 10, 8855])
Epoch [1/1], Step [12721/12942], Loss: 2.2130, Perplexity: 9.1428torch.Size([32, 16, 512])
torch.Size([32, 16, 8855])
Epoch [1/1], Step [12722/12942], Loss: 2.3267, Perplexity: 10.2441torch.Size([32, 15, 512])
torch.Size([32, 15, 8855])
Epoch [1/1], Step [12723/12942], Loss: 2.1698, Perplexity: 8.7564torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [12724/12942], Loss: 1.9204, Perplexity: 6.8239torch.Size([32, 15, 512])
torch.Size([32, 15, 8855])
Epoch [1/1], Step [12725/12942], Loss: 1.9803, Perplexity: 7.2447torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [12726/12942], Loss: 2.0552, Perplexity: 7.8083torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12727/12942], Loss: 1.9049, Perplexity: 6.7187torch.Size([32, 15, 512])
torch.Size([32, 15, 8855])
Epoch [1/1], Step [12728/12942], Loss: 2.3616, Perplex

Epoch [1/1], Step [12789/12942], Loss: 2.5921, Perplexity: 13.3579torch.Size([32, 15, 512])
torch.Size([32, 15, 8855])
Epoch [1/1], Step [12790/12942], Loss: 2.3664, Perplexity: 10.6587torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [12791/12942], Loss: 2.3328, Perplexity: 10.3066torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12792/12942], Loss: 2.1056, Perplexity: 8.2124torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [12793/12942], Loss: 1.9117, Perplexity: 6.7648torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12794/12942], Loss: 2.0558, Perplexity: 7.8130torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12795/12942], Loss: 1.9907, Perplexity: 7.3206torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [12796/12942], Loss: 2.0677, Perplexity: 7.9064torch.Size([32, 15, 512])
torch.Size([32, 15, 8855])
Epoch [1/1], Step [12797/12942], Loss: 2.1311, Perple

Epoch [1/1], Step [12858/12942], Loss: 2.0952, Perplexity: 8.1269torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [12859/12942], Loss: 2.1010, Perplexity: 8.1740torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [12860/12942], Loss: 2.2430, Perplexity: 9.4218torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [12861/12942], Loss: 2.0467, Perplexity: 7.7424torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12862/12942], Loss: 2.1260, Perplexity: 8.3813torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12863/12942], Loss: 1.9379, Perplexity: 6.9443torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [12864/12942], Loss: 2.2207, Perplexity: 9.2139torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12865/12942], Loss: 2.1188, Perplexity: 8.3213torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [12866/12942], Loss: 2.1396, Perplexit

Epoch [1/1], Step [12927/12942], Loss: 1.8045, Perplexity: 6.0768torch.Size([32, 13, 512])
torch.Size([32, 13, 8855])
Epoch [1/1], Step [12928/12942], Loss: 2.0989, Perplexity: 8.1571torch.Size([32, 11, 512])
torch.Size([32, 11, 8855])
Epoch [1/1], Step [12929/12942], Loss: 2.3900, Perplexity: 10.9135torch.Size([32, 18, 512])
torch.Size([32, 18, 8855])
Epoch [1/1], Step [12930/12942], Loss: 2.3865, Perplexity: 10.8749torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [12931/12942], Loss: 2.0397, Perplexity: 7.6882torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [12932/12942], Loss: 2.4289, Perplexity: 11.3460torch.Size([32, 14, 512])
torch.Size([32, 14, 8855])
Epoch [1/1], Step [12933/12942], Loss: 2.1313, Perplexity: 8.4260torch.Size([32, 16, 512])
torch.Size([32, 16, 8855])
Epoch [1/1], Step [12934/12942], Loss: 2.5094, Perplexity: 12.2972torch.Size([32, 12, 512])
torch.Size([32, 12, 8855])
Epoch [1/1], Step [12935/12942], Loss: 2.1501, Perpl

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [4]:
# (Optional) TODO: Validate your model.